<a href="https://colab.research.google.com/github/kaushik3012/lohum-assignment/blob/main/Copy_of_Lohum_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Part 1 - Feature Extraction

###Setup

Downloading the dataset

Importing the important libraries and the dataset

In [1]:
!wget -O Part1.csv https://raw.githubusercontent.com/shubham21155102/lohum_assignment_data_science/main/data/Part1.csv

--2023-11-06 03:29:53--  https://raw.githubusercontent.com/shubham21155102/lohum_assignment_data_science/main/data/Part1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
OpenSSL: error:0A000126:SSL routines::unexpected eof while reading
Unable to establish SSL connection.


In [2]:
import pandas as pd

df = pd.read_csv("Part1.csv")

EmptyDataError: No columns to parse from file

In [ ]:
df.head()

###1) Cleaning the Data

Remove missing values

In [ ]:
df = df.dropna()

In [ ]:
df.shape

Removing the Outliers

In [ ]:
#This function removes the value which lie outside the 25th and 75th percentile of the dataset
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

df = remove_outlier(df, 'soc')
df = remove_outlier(df, 'voltage')

Parse the time stamps column

In [ ]:
df['time_parsed'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S')
df = df.sort_values(by=['time_parsed'])   #Sorting the DataFrame according to the timestamp

###2) Algorithm to obtain the FECs that the given battery has undergone

Creating a charge status column which finds the change in charge
The charge status is negative if the battery is discharging and positive if it is charging.

In [ ]:
df['charge_status'] = df['soc'] - df['soc'].shift(1)

Creating a charge_current column which gives the current difference while charging. The charge_current value is 0 if the battery is discharging.

In [ ]:
df['charge_current']=0

charge_status = -1
for i in range(df.shape[0]):
  if(df['charge_status'].iloc[i]!=0):
    charge_status = df['charge_status'].iloc[i]

  if(charge_status>0):
    df['charge_current'].iloc[i] = df['current'].iloc[i]
  else:
    df['charge_current'].iloc[i] = 0


In [ ]:
df.head(1000)

The following expression finds the fraction of cycle completed at each timestep by adding only the positive values of charge_status and then dividing them by 100

In [ ]:
df['cycles'] = df['charge_status'].map(lambda x : x if x>0 else 0).cumsum()/100

In [ ]:
df.head(1000)

The Total FECs is given by the following code

In [ ]:
int(df['cycles'].iloc[-1])

###3) Capacity of Battery

Calculating the time difference

In [ ]:
df['time_secs'] = df['time_parsed'] - df['time_parsed'].shift(1)
df['time_secs'] = df['time_secs'].map(lambda x : (pd.Timedelta(x, unit='h').total_seconds())).fillna(0)

In [ ]:
df.head(1000)

Finding the Change of Charge at each timestamp (Q = I*t)

In [ ]:
df['charge_change'] = df['charge_current']*(df['time_secs']/3600)
df.head(1000)

Finding the index where 10th cycle ends

In [ ]:
cycle_10th=0
for i in range(df.shape[0]):
  if(df['cycles'].iloc[i]>=10):
    cycle_10th = i
    break

Following expression gives the Capacity of the battery (in Ah)

In [ ]:
# Summing all the Positive change of charges at all timestamps till the 10th cycle 
# and then divide by 10 to obtain the average over 10 cycles

df.head(cycle_10th)['charge_change'].sum()/10     

#Part 2 - Regression

###Setup

Loading the Dataset

In [6]:
!wget -O Part2.csv.xlsx https://raw.githubusercontent.com/shubham21155102/lohum_assignment_data_science/main/data/Part2.csv.xlsx

--2023-11-06 03:36:39--  https://raw.githubusercontent.com/shubham21155102/lohum_assignment_data_science/main/data/Part2.csv.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 

200 OK
Length: 44623 (44K) [application/octet-stream]
Saving to: ‘Part2.csv.xlsx’

Part2.csv.xlsx      100%[===================>]  43.58K  --.-KB/s    in 0.02s   

2023-11-06 03:36:40 (1.91 MB/s) - ‘Part2.csv.xlsx’ saved [44623/44623]



Importing the data and library

In [36]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import  train_test_split
import numpy as np
from sklearn.metrics import mean_absolute_error,mean_squared_error
df2=pd.DataFrame(pd.read_excel("Part2.xlsx"))
# df2 = pd.DataFrame(pd.read_excel("Part2.xlsx"))

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
df2.head()

Creating SOH column as per the given definition

In [ ]:
initial_capacity = df2['Discharge_Capacity (Ah)'].iloc[0]
df2['SOH'] = df2['Discharge_Capacity (Ah)']/initial_capacity
df2.head()

###Cleaning the data

Sorting the DataFrame according to the Cycle Index

In [ ]:
df2 = df2.sort_values(by=['Cycle_Index'])

Plotting our dataset

In [ ]:
plt.scatter(df2['Cycle_Index'], df2['SOH'])
plt.show()

We can see from the above plot that there are some outliers in the data, 
So we'll proceed to remove these outliers

In [ ]:
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

df2 = remove_outlier(df2, 'SOH')

Visualizing the data again

In [ ]:
plt.scatter(df2['Cycle_Index'], df2['SOH'])
plt.show()

###1) Modelling the SOH of the cell as a function of cycle number

Preprocessing the data and splitting them into training and test datasets

In [ ]:

X = np.array(df2['Cycle_Index']).reshape(-1, 1)
y = np.array(df2['SOH']).reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

Creating a learning regression model using Scikit-learn library and training it on the train dataset

In [ ]:
regr = LinearRegression()
  
regr.fit(X_train, y_train)

Testing the model on test dataset

In [ ]:
print(regr.score(X_test, y_test))

Plotting the results predicted by the model (red line)

In [ ]:
y_pred = regr.predict(X_test)
plt.scatter(X_test, y_test, color ='b')
plt.plot(X_test, y_pred, color ='r')
plt.xlabel('Cycle_Index')
plt.ylabel('SOH')
plt.show()

Creating the predict_SOH function as per the question

In [ ]:
def predict_SOH(df_in):
  X_in = np.array(df_in['Cycle_Index']).reshape(-1, 1)
  pred_SOH = regr.predict(X_in)
  return pred_SOH

print(predict_SOH(df2))

###2) Obtain RUL

Predict after how many cycles model reaches 80% SOH

In [ ]:
life_cycle = np.array(0).reshape(1,-1)
y=0
while(True):
  y = regr.predict(life_cycle)
  if(y<=0.8):     #Count cycles until SOH decreases to 80%
    break
  life_cycle+=1
print(life_cycle[0][0])

RUL after the following Cycles: 500, 1000, 2000, 3000 and 3500

In [ ]:
rul_500 = life_cycle[0][0] - 500
rul_1000 = life_cycle[0][0] - 1000
rul_2000 = life_cycle[0][0] - 2000
rul_3000 = life_cycle[0][0] - 3000
rul_3500 = life_cycle[0][0] - 3500

print("RUL at 500th Cycle : ",rul_500)
print("RUL at 1000th Cycle : ",rul_1000)
print("RUL at 2000th Cycle : ",rul_2000)
print("RUL at 3000th Cycle : ",rul_3000)
print("RUL at 3500th Cycle : ",rul_3500)

### 3) Evaluate the performance of the model

Calculating the error estimates using Scikit-learn module

In [ ]:
#This function estimates the MAE and RMSE errors between the Actual and predicted SOH values using the Scikit-learn libraries
def Estimate_error(df_in):
  y_test = df_in['SOH']
  y_pred = predict_SOH(df_in)
  mae = mean_absolute_error(y_true=y_test,y_pred=y_pred)
  rmse = mean_squared_error(y_true=y_test,y_pred=y_pred,squared=False)
  return (rmse, mae)

In [ ]:
rmse,mae = Estimate_error(df2)

Print the performance metrics

In [ ]:
print("MAE:",mae)
print("RMSE:",rmse)

In [ ]:
initial_capacity = df2['Discharge_Capacity (Ah)'].iloc[0]
df2['SOH'] = df2['Discharge_Capacity (Ah)']/initial_capacity

print(Estimate_error(df2))

#Summary

###Part 1 - Feature Extraction

Task 1


---

In this initial section, I have performed preprocessing of data. <br>
I used the dropna function available in pandas library to drop the null values.<br>
I have used a custom function to remove the outliers present in the data by excluding those values which lie outside the inter-quartile range.
<br>
I have also parsed the timestamps so that I can sort the table according to the time and it remains consistent with chronology of the battery charging and discharging.


Task 2 

---

In this section I have written an algorithm to find the full equivalent cycles of the battery. I created a charge status column which monitors the change in the SOC. Then I have created a charge_current column which monitors the change in charging current of the battery.
<br>
After that I create a cycles column which captures the cycles completed at a particular timestamp. This is calculated by adding all the charge_status cummulatively and then dividing them by 100. In this way, the last value gives the total FEC completed by the battery.

Task 3

---

Initially, I have calculated the change in time at each step. Now, I multiplt each of the time change with the current change to obtain the change in charge. Then, I have found the index where 10th cycle ends iteratively. Finally, I summed the change in charges over 10 cycles and divided them by 10 to get the average Capacity of the battery.


###Part 2 - Regression

Task 1 

---

Before starting I have cleaned the dataset provided and removed the outliers by the same method used in previous task. I have started by preprocessing the data and then split the whole dataset into training and testing data. In this process I have used the Scikit-learn library and the numpy library. After that I have used the Scikit-learn's LinearRegression funtion to create and train a linear regression model. I have chosen a linear regression model because in while visualing the data, it seemed to resemble a straight line and hence, intuitively should give better results. 
Training the model on 75% of the dataset took a time of around 1 second. The inputs and targets fed are Cycle_index and SOH columns of the DataFrame.
<br>
After that I have tested the model on the test dataset and plotted  the comparison between the actual and predicted results. Finally, I have created the predict_SOH function as asked in the question.


Task 2 

---

In this section I have calculated the remaining useful life (RUL) of the battery. First, I have tried to find the total number of cycles which is useful in its lifetime. So I thought of an algorithm where we count the cycles until SOH decreases to 80%.
So, in this way the RUL comes out to be :
<br>RUL at 500th Cycle :  3108
<br>RUL at 1000th Cycle :  2608
<br>RUL at 2000th Cycle :  1608
<br>RUL at 3000th Cycle :  608
<br>RUL at 3500th Cycle :  108

Task 3 

---

In this section, I have calculated the performance metrics of my model. The metrics used are MAE and RMSE. These are estimated errors calculated using Scikit's functions.<br>
The values come out to be :<br>
MAE: 0.0061345402596345265<br>
RMSE: 0.007837251920956927